# Solucion al Problema del vendedor ambulante mediante algoritmos geneticos


## Introduccion

**El problema del vendedor ambulante** (TSP) consiste en, "Dada una lista de ciudades y las distancias entre cada par de ciudades, ¿cuál es la ruta más corta posible que visita cada ciudad exactamente una vez y vuelve a la ciudad de origen?".

En este caso, un vendedor debe hacer un recorrido por las siguientes ciudades de Colombia en su carro (no necesariamente en este orden):



*   Palmira;

> Indented block

> Indented block




*   Pasto;
*   Tuluá;
*   Bogota;
*   Pereira;
*   Armenia;
*   Manizales;
*   Valledupar;
*   Montería;
*   Soledad;
*   Cartagena;
*   Barranquilla;
*   Medellín;
*   Bucaramanga;
*   Cúcuta.

Donde el costo de desplazamiento entre ciudades es la suma del valor de la hora del vendedor (es un parámetro que debe estudiarse), el costo de los peajes y el costo del combustible. Cada equipo debe definir en qué carro hace el recorrido el vendedor y de allí extraer el costo del combustible.


**Restricciones**

Una solución al TSP tiene que tener:

*   El viajero comienza y termina en la misma ciudad.
*   El viajero no vuelve a visitar ninguna ciudad intermedia, por lo que cada una de las ciudades de la solución debe ser única.
*   El viajero visita las 14 + 1 (para volver) = 15 ciudades.

**Algoritmos Geneticos**

Estos algoritmos hacen evolucionar una población de individuos sometiéndola a acciones aleatorias, semejantes a las que actúan en la evolución biológica (mutaciones y recombinaciones genéticas), así como también a una selección. De acuerdo con algún criterio, se decide cuáles son los individuos más adaptados, que sobreviven, y cuáles son los menos aptos, que son descartados.

Los algoritmos genéticos se enmarcan dentro de los algoritmos evolutivos, que incluyen también las estrategias evolutivas, la programación evolutiva y la programación genética.

La implementación de un algoritmo genético imita los procesos biológicos y no es adecuada para problemas restrictivos. Esto significa que las mutaciones son aleatorias (por lo que podrían causar duplicación, rompiendo el punto 2 anterior), los cruces entre soluciones también son aleatorios (por lo que podrían romper el punto 1) y las supresiones también son posibles mutaciones en las que un gen podría ser eliminado por completo (rompiendo el punto 3). Entonces, ¿cómo pueden modificarse y desarrollarse para el TSP?

**Metodología**

Para construir el algoritmo genético se pueden aprovechar paquetes existentes como NumPy y PyGAD. PyGAD es un paquete dedicado a Algoritmos Genéticos escrito en Python. Es realmente útil y tiene muchos casos de uso, la mayoría orientados al entrenamiento de redes neuronales. Esto significa que no se ha adaptado completamente a la resolución del TSP. Sin embargo, tiene opciones de personalización y parámetros muy intuitivos a medida que se construye una instancia de algoritmo genético. Que se exploran a continuación para que sea más a medida para el TSP restrictiva.

**Construcción del algoritmo genético**

Un algoritmo genético se compone de un número determinado de partes:

1. Una población inicial
2. Una función de aptitud
3. Una operación de cruce
4. Una operación de mutación

Estos constituyentes en el caso de esta solución para fijar el algoritmo genético para el TSP restrictivo serán:

1. Una población de soluciones generadas programáticamente con un orden aleatorio de los almacenes empezando y terminando en un almacén dado
2. Una función para determinar la distancia entre cada tienda en una solución
3. Una función para imitar la reproducción en un proceso natural, en el que partes de dos soluciones muy adecuadas se transmiten a los hijos, utilizando el operador de cruce de coincidencia parcial que mantiene las restricciones.
4. Una función para cambiar aleatoriamente un elemento de cada solución para imitar las mutaciones en los procesos naturales - una inversión que de nuevo mantiene las restricciones.

También existe la selección de padres, pero de esto se encarga PyGAD. Ahora es importante asegurarnos de que nuestra biología está a la altura aquí también y podemos empezar a sustituir algunas de las frases más largas de arriba por otras más cortas y precisas:

**gen** - un almacén a visitar en una solución dada

**Cromosoma** - conjunto de almacenes unidos para formar una solución.

**población** - el conjunto de cromosomas


## Implementacion

In [2]:
#!pip install --upgrade geopandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 66.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 77.9 MB/s eta 0:00:00


In [37]:
#!pip install pygad==2.17

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.2/55.2 KB 3.4 MB/s eta 0:00:00


In [17]:
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd

from shapely.geometry import Point

# Coordenadas de las ciudades en colombia
df = pd.DataFrame({
    'City': ['Palmira', 'Pasto', 'Tuluá', 'Bogota', 'Pereira','Armenia','Manizales','Valledupar','Montería','Soledad','Cartagena','Barranquilla','Medellín','Bucaramanga','Cúcuta'],
    'Country': ['Colombia', 'Colombia', 'Colombia', 'Colombia', 'Colombia', 'Colombia', 'Colombia', 'Colombia', 'Colombia', 'Colombia', 'Colombia', 'Colombia', 'Colombia', 'Colombia', 'Colombia'],
    'latitude': [3.53944, 1.21361, 4.08466, 4.60971, 4.81333,4.53389,5.06889,10.46314,8.74798 ,10.91843,10.39972,10.96854,6.25184,7.12539,7.89391],
    'longitude': [-76.30361, -77.28111, -76.19536, -74.08175, -75.69611,-75.68111,-75.51738,-73.25322,-75.88143,-74.76459,-75.51444,-74.78132,-75.56359,-73.1198,-72.50782]
})

df.head()

,City,Country,latitude,longitude
0,Palmira,Colombia,3.53944,-76.30361
1,Pasto,Colombia,1.21361,-77.28111
2,Tuluá,Colombia,4.08466,-76.19536
3,Bogota,Colombia,4.60971,-74.08175
4,Pereira,Colombia,4.81333,-75.69611


Mediante folium se ubica a Colombia y luego se le dibujan cada una de las ubicaciones a donde tendra que viajar el vendendor.

In [18]:
import folium
# Las coordenadas (4.570868, -74.297333) pertenecen a Colombia
map = folium.Map(location=[4.570868, -74.297333], zoom_start=6, tiles="stamentoner")

In [19]:
# Metodo para marcar en el mapa cada una de las ciudades
for _, r in df.iterrows():
  folium.Marker(
      [r['latitude'], r['longitude']], popup=f'<i>{r["City"]}</i>'
  ).add_to(map)

In [20]:
map

**Comprobación de la metodología de la distancia**

Para evaluar la calidad de cada solución, es necesario disponer de una medida de adecuación. En este ejemplo se ha utilizado la distancia "a vuelo de pájaro" sin tener en cuenta las distancias reales por carretera.

In [31]:
from geopy.distance import geodesic
# Se mide la distancia entre algunas ciudades para comparar

#Palmira
origin = (df['latitude'][0], df['longitude'][0])
#Pasto
dest1 = (df['latitude'][1], df['longitude'][1])
#Tulua
dest2 = (df['latitude'][2], df['longitude'][2])

A = geodesic(origin, dest1).kilometers
B = geodesic(origin, dest2).kilometers

print('La distancia entre Palmira y Pasto: ' , A)
print('La distancia entre Palmira y Tulua: ' , B)

La distancia entre Palmira y Pasto:  279.21546355148536
La distancia entre Palmira y Tulua:  61.477267778467635


##Definición de funciones

El algoritmo requiere un conjunto de funciones predefinidas, ya que el algoritmo genético "out of the box" no soporta un TSP.

1. build_population: construye una población de cromosomas para probar con las restricciones apropiadas aplicadas.
2. fitness_func: Se utiliza para probar una solución para ver qué tan bien se desempeña, en este caso el fitness_func se evaluará sobre la base de la distancia a vuelo de pájaro entre cada punto sucesivo
3. pmx_crossover: realiza el cruce de un padre y un hijo con la lógica Partially Matched Crossover (PMX).
4. crossover_func: aplica el cruce
5. on_crossover: aplica la mutación después del cruce
6. on_generation: se utiliza para imprimir el progreso y los resultados en cada generación

In [32]:
import random
import numpy as np


def build_population(size, chromosome_size):
  population = []
  for i in range(size):
    home_city = 0
    added = {home_city:'Added'}
    chromosome = [home_city]

    while len(chromosome) < chromosome_size:
      proposed_gene = random.randint(0, chromosome_size-1)
      if added.get(proposed_gene) is None:
        chromosome.append(proposed_gene)
        added.update({proposed_gene:'Added'})
      else:
        pass

    chromosome.append(home_city)

    population.append(chromosome)

  return np.array(population)

population = build_population(100, 10)
population.shape

(100, 11)

In [39]:
def fitness_func(solution, solution_idx):
  # loop through the length of the chromosome finding the distance between each
  # gene added 

  # to increment
  total_dist = 0

  for gene in range(0, len(solution)):

    # get the lon lat of the two points
    a = genes.get(stores[solution[gene]])
    
    try:
      b = genes.get(stores[solution[gene + 1]])

      # find the distance (crow flies)
      dist = geodesic(a, b).kilometers

    except IndexError:
      dist = 0

    total_dist += dist

  # to optimise this value in the positive direction the inverse of dist is used
  fitness = 1 / total_dist

  return fitness 

In [33]:
def pmx_crossover(parent1, parent2, sequence_start, sequence_end):
  # initialise a child
  child = np.zeros(parent1.shape[0])

  # get the genes for parent one that are passed on to child one
  parent1_to_child1_genes = parent1[sequence_start:sequence_end]

  # get the position of genes for each respective combination
  parent1_to_child1 =  np.isin(parent1,parent1_to_child1_genes).nonzero()[0]

  for gene in parent1_to_child1:
    child[gene] = parent1[gene]

  # gene of parent 2 not in the child
  genes_not_in_child = parent2[np.isin(parent2, parent1_to_child1_genes, invert=True).nonzero()[0]]
  
  # if the gene is not already
  if genes_not_in_child.shape[0] >= 1:
    for gene in genes_not_in_child:
      if gene >= 1:
        lookup = gene
        not_in_sequence = True

        while not_in_sequence:
          position_in_parent2 = np.where(parent2==lookup)[0][0]

          if position_in_parent2 in range(sequence_start, sequence_end):
            lookup = parent1[position_in_parent2]

          else:
            child[position_in_parent2] = gene
            not_in_sequence = False

  return child

In [25]:
def crossover_func(parents, offspring_size, ga_instance):
  offspring = []
  idx = 0
  while len(offspring) != offspring_size[0]:

    # locate the parents
    parent1 = parents[idx % parents.shape[0], :].copy()
    parent2 = parents[(idx + 1) % parents.shape[0], :].copy()

    # find gene sequence in parent 1 
    sequence_start = random.randint(1, parent1.shape[0]-4)
    sequence_end = random.randint(sequence_start, parent1.shape[0]-1)

    # perform crossover
    child1 = pmx_crossover(parent1, parent2, sequence_start, sequence_end)
    child2 = pmx_crossover(parent2, parent1, sequence_start, sequence_end)    

    offspring.append(child1)
    offspring.append(child2)


    idx += 1

  return np.array(offspring)

In [26]:
def mutation_func(offspring, ga_instance):

  for chromosome_idx in range(offspring.shape[0]):
    # define a sequence of genes to reverse
    sequence_start = random.randint(1, offspring[chromosome_idx].shape[0] - 2)
    sequence_end = random.randint(sequence_start, offspring[chromosome_idx].shape[0] - 1)
    
    genes = offspring[chromosome_idx, sequence_start:sequence_end]

    # start at the start of the sequence assigning the reverse sequence back to the chromosome
    index = 0
    if len(genes) > 0:
      for gene in range(sequence_start, sequence_end):

          offspring[chromosome_idx, gene] = genes[index]

          index += 1

    return offspring

In [34]:
def on_crossover(ga_instance, offspring_crossover):
    # apply mutation to ensure uniqueness 
    offspring_mutation  = mutation_func(offspring_crossover, ga_instance)

    # save the new offspring set as the parents of the next generation
    ga_instance.last_generation_offspring_mutation = offspring_mutation

In [35]:
def on_generation(ga):
    print("Generation", ga.generations_completed)
    print(ga.population)

##Ejecución del algoritmo

El algoritmo genético se configura como instancia y en la inicialización se dan varios parámetros.

A continuación, el algoritmo se ejecuta para encontrar la mejor solución durante un número determinado de generaciones.

In [38]:
import pygad

In [41]:
genes = {store_num:[lat, lon] for store_num, lat, lon in zip(df['City'], df['latitude'], df['longitude'])}
stores = list(genes.keys())
len(stores)

15

In [44]:
population = build_population(100, 15)
len(population[0])

16

In [45]:
def fitness_func(solution, solution_idx):
  # loop through the length of the chromosome finding the distance between each
  # gene added 

  # to increment
  total_dist = 0

  for gene in range(0, len(solution)):

    # get the lon lat of the two points
    a = genes.get(stores[solution[gene]])
    
    try:
      b = genes.get(stores[solution[gene + 1]])

      # find the distance (crow flies)
      dist = geodesic(a, b).kilometers

    except IndexError:
      dist = 0

    total_dist += dist

  # to optimise this value in the positive direction the inverse of dist is used
  fitness = 1 / total_dist

  return fitness 

In [46]:
def pmx_crossover(parent1, parent2, sequence_start, sequence_end):
  # initialise a child
  child = np.zeros(parent1.shape[0])

  # get the genes for parent one that are passed on to child one
  parent1_to_child1_genes = parent1[sequence_start:sequence_end]

  # get the position of genes for each respective combination
  parent1_to_child1 =  np.isin(parent1,parent1_to_child1_genes).nonzero()[0]

  for gene in parent1_to_child1:
    child[gene] = parent1[gene]

  # gene of parent 2 not in the child
  genes_not_in_child = parent2[np.isin(parent2, parent1_to_child1_genes, invert=True).nonzero()[0]]
  
  if genes_not_in_child.shape[0] >= 1:
    for gene in genes_not_in_child:
      if gene >= 1:
        lookup = gene
        not_in_sequence = True

        while not_in_sequence:
          position_in_parent2 = np.where(parent2==lookup)[0][0]

          if position_in_parent2 in range(sequence_start, sequence_end):
            lookup = parent1[position_in_parent2]

          else:
            child[position_in_parent2] = gene
            not_in_sequence = False

  return child

In [47]:
def crossover_func(parents, offspring_size, ga_instance):
  offspring = []
  idx = 0
  while len(offspring) != offspring_size[0]:

    # locate the parents
    parent1 = parents[idx % parents.shape[0], :].copy()
    parent2 = parents[(idx + 1) % parents.shape[0], :].copy()

    # find gene sequence in parent 1 
    sequence_start = random.randint(1, parent1.shape[0]-4)
    sequence_end = random.randint(sequence_start, parent1.shape[0]-1)

    # perform crossover
    child1 = pmx_crossover(parent1, parent2, sequence_start, sequence_end)
    child2 = pmx_crossover(parent2, parent1, sequence_start, sequence_end)
    

    offspring.append(child1)
    offspring.append(child2)

    idx += 1

  return np.array(offspring)

In [48]:
def mutation_func(offspring, ga_instance):

  for chromosome_idx in range(offspring.shape[0]):
    # define a sequence of genes to reverse
    sequence_start = random.randint(1, offspring[chromosome_idx].shape[0] - 2)
    sequence_end = random.randint(sequence_start, offspring[chromosome_idx].shape[0] - 1)
    
    genes = offspring[chromosome_idx, sequence_start:sequence_end]

    # start at the start of the sequence assigning the reverse sequence back to the chromosome
    index = 0
    if len(genes) > 0:
      for gene in range(sequence_start, sequence_end):

          offspring[chromosome_idx, gene] = genes[index]

          index += 1

    return offspring

In [49]:
def on_crossover(ga_instance, offspring_crossover):
    # apply mutation to ensure uniqueness 
    offspring_mutation  = mutation_func(offspring_crossover, ga_instance)

    # save the new offspring set as the parents of the next generation
    ga_instance.last_generation_offspring_mutation = offspring_mutation

In [50]:
def on_generation(ga):
    print("Generation", ga.generations_completed)
    print(ga.population)

In [51]:
ga_instance = pygad.GA(num_generations=100,
                       num_parents_mating=40,
                       fitness_func=fitness_func,
                       sol_per_pop=200,
                       initial_population=population,
                       gene_space=range(0, 15),
                       gene_type=int,
                       mutation_type=mutation_func,
                       on_generation=on_generation,
                       crossover_type=crossover_func, 
                       keep_parents=6,
                       mutation_probability=0.4)

In [52]:
ga_instance.run()

Generation 1
[[ 0  1  2 ...  5  3  0]
 [ 0  3  2 ...  5  1  0]
 [ 0  1  3 ...  8 12  0]
 ...
 [ 0  8  7 ...  6 13  0]
 [ 0  8  7 ...  6 13  0]
 [ 0  8  9 ... 12  1  0]]
Generation 2
[[ 0  3  2 ...  5  1  0]
 [ 0  3  2 ...  5  1  0]
 [ 0  8  7 ... 13  1  0]
 ...
 [ 0  1  2 ...  5  3  0]
 [ 0  1  2 ...  5  3  0]
 [ 0  2  3 ...  5  1  0]]
Generation 3
[[ 0  3  2 ...  5  1  0]
 [ 0  3  2 ...  5  1  0]
 [ 0  3  2 ...  5  1  0]
 ...
 [ 0  3  2 ...  5  1  0]
 [ 0  8  6 ...  4  1  0]
 [ 0  3 10 ... 12  1  0]]
Generation 4
[[0 4 2 ... 6 1 0]
 [0 1 2 ... 5 3 0]
 [0 3 2 ... 5 1 0]
 ...
 [0 3 2 ... 5 1 0]
 [0 3 2 ... 5 1 0]
 [0 3 2 ... 5 1 0]]
Generation 5
[[0 1 2 ... 6 4 0]
 [0 2 3 ... 5 1 0]
 [0 4 2 ... 6 1 0]
 ...
 [0 1 2 ... 5 3 0]
 [0 1 2 ... 5 3 0]
 [0 1 2 ... 5 3 0]]
Generation 6
[[0 1 2 ... 6 4 0]
 [0 1 2 ... 6 4 0]
 [0 2 4 ... 6 1 0]
 ...
 [0 2 3 ... 5 1 0]
 [0 4 2 ... 5 1 0]
 [0 2 3 ... 6 1 0]]
Generation 7
[[0 2 4 ... 5 1 0]
 [0 1 2 ... 5 4 0]
 [0 1 2 ... 6 4 0]
 ...
 [0 1 2 ... 6 4 0]


## Evaluacion de los resultados


In [53]:
solution, solution_fitness, solution_idx = ga_instance.best_solution()
print(f'Generation of best solution: {ga_instance.best_solution_generation}')
print("Fitness value of the best solution = {solution_fitness}".format(solution_fitness=solution_fitness))
print("Index of the best solution : {solution_idx}".format(solution_idx=solution_idx))

Generation of best solution: 6
Fitness value of the best solution = 0.00028840119894129495
Index of the best solution : 0


In [55]:
def verify_solution(solution, max_gene):
  if min(solution) != 0:
    print('Failed values below 0')

  if max(solution) != max_gene:
    print('Failed values less than or above max possible value')

  if len(set(solution)) - len(solution) != -1:
    print(len(set(solution)) - len(solution))
    print('Failed solution does not contain unique values')

In [56]:
verify_solution(solution, len(stores))
solution

Failed values less than or above max possible value


array([ 0,  2,  4,  8, 11,  9, 10,  7, 12, 13, 14,  3,  6,  5,  1,  0])

In [62]:
df.head(15)

,City,Country,latitude,longitude
0,Palmira,Colombia,3.53944,-76.30361
1,Pasto,Colombia,1.21361,-77.28111
2,Tuluá,Colombia,4.08466,-76.19536
3,Bogota,Colombia,4.60971,-74.08175
4,Pereira,Colombia,4.81333,-75.69611
5,Armenia,Colombia,4.53389,-75.68111
6,Manizales,Colombia,5.06889,-75.51738
7,Valledupar,Colombia,10.46314,-73.25322
8,Montería,Colombia,8.74798,-75.88143
9,Soledad,Colombia,10.91843,-74.76459


In [57]:
points = [genes.get(stores[id]) + [stores[id]] for id in solution]
points[:5]

[[3.53944, -76.30361, 'Palmira'],
 [4.08466, -76.19536, 'Tuluá'],
 [4.81333, -75.69611, 'Pereira'],
 [8.74798, -75.88143, 'Montería'],
 [10.96854, -74.78132, 'Barranquilla']]

In [60]:
map = folium.Map(location=[4.570868, -74.297333], zoom_start=6, tiles="stamentoner")

for point in range(0, len(points)):
  folium.Marker(
      [points[point][0], points[point][1]], popup=f'<i>{points[point][2]}</i>'
  ).add_to(map)

  try:
    folium.PolyLine([(points[point][0], points[point][1]), 
                      (points[point+1][0], points[point+1][1])],
                  color='red',
                  weight=5,
                  opacity=0.8).add_to(map)

  except IndexError:
    pass

In [61]:
map

## Referencias

https://en.wikipedia.org/wiki/Travelling_salesman_problem

https://es.wikipedia.org/wiki/Algoritmo_genético


##Evaluación de los resultados

La solución resultante puede comprobarse y analizarse utilizando la propia ga_instance